In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import csv
import nltk

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Load the CSV file
file_path = 'amzn_data.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(file_path)

# Drop rows with NaN values in 'body' or 'entities' columns
df = df.dropna(subset=['body', 'entities'])

# Extract the 'body' and 'entities' columns to match the sample_data format
sample_data = [(row['body'], row['entities']) for _, row in df.iterrows()]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ParserError: Error tokenizing data. C error: EOF inside string starting at row 76790

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Clean text
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'http\S+|www\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove @ mentions and # hashtags
    text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # Remove special characters
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

# Remove stop words
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

# Tokenize text
def tokenize(text):
    return word_tokenize(text)

# Remove specific words (e.g., 'tsla')
def remove_specific_words(tokens, words_to_remove):
    return [word for word in tokens if word not in words_to_remove]



# Apply preprocessing to the sample data
preprocessed_data = [(clean_text(text), label) for text, label in sample_data]
preprocessed_data = [(remove_stopwords(text), label) for text, label in preprocessed_data]
preprocessed_data = [(tokenize(text), label) for text, label in preprocessed_data]
preprocessed_data = [(remove_specific_words(text, ['amzn']), label) for text, label in preprocessed_data]

# Print preprocessed data
print("Preprocessed Data:")
for text, label in preprocessed_data:
    print(f"Text: {text}, Label: {label}")


# # Clean text
# def clean_text(text):
#     text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
#     text = re.sub(r'http\S+|www\S+', '', text, flags=re.MULTILINE)  # Remove URLs
#     text = re.sub(r'\@\w+|\#', '', text)  # Remove @ mentions and # hashtags
#     text = re.sub(r'[^A-Za-z0-9 ]+', '', text)  # Remove special characters
#     text = text.lower()  # Convert text to lowercase
#     text = re.sub(r'\d+', '', text)  # Remove digits
#     text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
#     return text

# # Remove stop words
# stop_words = set(stopwords.words('english'))

# def remove_stopwords(text):
#     return ' '.join([word for word in text.split() if word not in stop_words])

# # Tokenize text
# def tokenize(text):
#     return word_tokenize(text)

# # Apply preprocessing to the sample data
# preprocessed_data = [(clean_text(text), label) for text, label in sample_data]
# preprocessed_data = [(remove_stopwords(text), label) for text, label in preprocessed_data]
# preprocessed_data = [(tokenize(text), label) for text, label in preprocessed_data]

# # Print preprocessed data
# print("Preprocessed Data:")
# for text, label in preprocessed_data:
#     print(f"Text: {text}, Label: {label}")


Streaming output truncated to the last 5000 lines.
Text: ['tsla', 'next', 'stop'], Label: Bullish
Text: ['tsla', 'way'], Label: Bullish
Text: ['tsla', 'goooo'], Label: Bullish
Text: ['tsla', 'good', 'ol', 'tesla', 'stuck'], Label: Bullish
Text: ['tsla', 'intraday', 'bottom', 'thus', 'far'], Label: Bullish
Text: ['tsla', 'batteries', 'new', 'oil', 'tesla', 'new', 'saudi', 'arabia'], Label: Bullish
Text: ['tsla', 'ramping', 'resistance', 'takeout', 'week'], Label: Bullish
Text: ['tsla', 'ready'], Label: Bullish
Text: ['tsla', 'moves', 'one', 'point', 'bears', 'post', 'sell', 'late', 'chance'], Label: Bullish
Text: ['tsla', 'nice', 'parabolic', 'pattern', 'back'], Label: Bullish
Text: ['tsla', 'like', 'steele', 'hands'], Label: Bullish
Text: ['tsla'], Label: Bullish
Text: ['tsla', 'q', 'delivers', 'huge'], Label: Bullish
Text: ['tsla', 'long', 'another', 'quiet', 'little', 'stock', 'adom', 'er', 'today'], Label: Bullish
Text: ['tsla', 'close', 'today'], Label: Bullish
Text: ['tsla', 'gett

In [ ]:

# Convert texts back to strings for vectorization
texts = [' '.join(text) for text, label in preprocessed_data]
labels = [label for text, label in preprocessed_data]

# Vectorize texts
vectorizer = CountVectorizer()
word_counts = vectorizer.fit_transform(texts)
tfidf_transformer = TfidfTransformer()
tfidf_matrix = tfidf_transformer.fit_transform(word_counts)

print("\nTF-IDF Matrix:")
print(tfidf_matrix.toarray())



TF-IDF Matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:

# Aggregate TF-IDF scores by sentiment
def aggregate_tfidf_by_sentiment(tfidf_matrix, labels, vectorizer):
    feature_names = vectorizer.get_feature_names_out()
    sentiment_tfidf = {'Bullish': Counter(), 'Bearish': Counter()}

    for i, label in enumerate(labels):
        tfidf_vector = tfidf_matrix[i]
        for word, tfidf_value in zip(feature_names, tfidf_vector.toarray()[0]):
            sentiment_tfidf[label][word] += tfidf_value

    return sentiment_tfidf

# Aggregate the TF-IDF scores
sentiment_tfidf = aggregate_tfidf_by_sentiment(tfidf_matrix, labels, vectorizer)

print("\nAggregated TF-IDF Scores by Sentiment:")
print(sentiment_tfidf)



Aggregated TF-IDF Scores by Sentiment:
{'Bullish': Counter({'tesla': 229.831422902618, 'today': 206.9759176499149, 'bears': 204.9223867106151, 'gme': 202.66878179980608, 'tomorrow': 191.03488840541704, 'go': 190.35131446030763, 'buy': 187.58910369760258, 'get': 163.42705225964335, 'lets': 160.93507034399036, 'aapl': 158.34398152622305, 'nio': 152.02326240228496, 'going': 151.08458710117168, 'next': 145.67268536531793, 'earnings': 144.28042271282692, 'week': 141.15654315259715, 'amc': 139.58602556120437, 'day': 138.59377208214343, 'like': 137.40975627841908, 'spy': 129.28432259355563, 'good': 122.65580017635435, 'elon': 122.23655859988793, 'lol': 120.12705983948862, 'back': 120.00023946929693, 'stock': 118.998091837812, 'see': 118.17151590870591, 'market': 113.03595984999039, 'dont': 109.33382631394005, 'time': 106.76789462834093, 'green': 100.73986415953028, 'im': 96.4903642481171, 'calls': 95.34045596985824, 'short': 94.31879834994878, 'coming': 92.72369033216215, 'shorts': 92.663354

In [ ]:


# Categorize words by sentiment
def categorize_words(labeled_texts):
    sentiment_words = {'Bullish': Counter(), 'Bearish': Counter()}
    for text, sentiment in labeled_texts:
        sentiment_words[sentiment].update(text)
    return sentiment_words

# Categorize and create sentiment dictionary
sentiment_words = categorize_words(preprocessed_data)

# Create sentiment dictionary using aggregated TF-IDF scores and word frequencies
def create_enhanced_sentiment_dict(sentiment_tfidf, sentiment_words):
    enhanced_sentiment_dict = {}
    for sentiment, words in sentiment_words.items():
        for word, freq in words.items():
            tfidf_score = sentiment_tfidf[sentiment].get(word, 0)
            if sentiment == 'Bullish':
                enhanced_sentiment_dict[word] = enhanced_sentiment_dict.get(word, 0) + (tfidf_score * freq)
            elif sentiment == 'Bearish':
                enhanced_sentiment_dict[word] = enhanced_sentiment_dict.get(word, 0) - (tfidf_score * freq)
    return enhanced_sentiment_dict

# Create the enhanced sentiment dictionary
enhanced_sentiment_dict = create_enhanced_sentiment_dict(sentiment_tfidf, sentiment_words)

In [ ]:

# # Normalize the sentiment dictionary
# def normalize_sentiment_dict(sentiment_dict):
#     max_score = max(abs(score) for score in sentiment_dict.values())
#     for word in sentiment_dict:
#         sentiment_dict[word] /= max_score
#     return sentiment_dict

# enhanced_sentiment_dict = normalize_sentiment_dict(enhanced_sentiment_dict)

# print("\nEnhanced Sentiment Dictionary:")
# print(enhanced_sentiment_dict)



Enhanced Sentiment Dictionary:
{'tsla': 1.0, 'lol': 0.0016809981796824202, 'looks': 0.00022254832024289618, 'like': 0.0036559939712158644, 'rush': 2.2978545074140237e-06, 'buy': 0.006026535698775157, 'resistance': 0.00012200103654394552, 'maybe': 8.69427249102259e-05, 'getting': 0.0002649934789677749, 'ready': 0.0002372643108833427, 'wipe': 3.3745506066435645e-07, 'shirts': 4.783345967095936e-07, 'still': 0.000839866597396499, 'bargain': 7.351233396010112e-07, 'price': 0.0007398391412808179, 'going': 0.0037773977994299645, 'robinhood': 9.947935054973711e-06, 'market': 0.002469751100452686, 'order': 4.2266243579767127e-05, 'wont': 6.0642274303577695e-05, 'even': 0.0003690222371274728, 'go': 0.005013233873397638, 'thru': 3.716349427575079e-06, 'minutes': 1.7601885520610638e-05, 'pending': 1.3412065069493177e-06, 'gme': 0.007292674403987321, 'pushing': 9.992126027174372e-06, 'come': 0.000972265441382313, 'baby': 0.00019933323687805542, 'keep': 0.00039898274303963236, 'recovering': 1.2779

In [ ]:

# Write the enhanced sentiment dictionary to a CSV file
csv_file_path = 'TESLA_dictionary.csv'  # Specify your desired file path

with open(csv_file_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Word', 'Sentiment_Score'])  # Write header row
    for word, score in enhanced_sentiment_dict.items():
        writer.writerow([word, score])

print(f"\nEnhanced sentiment dictionary has been written to {csv_file_path}")



Enhanced sentiment dictionary has been written to TESLA_dictionary.csv


In [ ]:

# Calculate sentiment scores using enhanced sentiment dictionary for Sample comments...
def sentiment_scores_with_tfidf_enhanced(texts, sentiment_dict, tfidf_matrix, vectorizer):
    scores = []
    feature_names = vectorizer.get_feature_names_out()
    for i, text in enumerate(texts):
        score = 0
        tfidf_vector = tfidf_matrix[i]
        for word, tfidf_value in zip(feature_names, tfidf_vector.toarray()[0]):
            score += sentiment_dict.get(word, 0) * tfidf_value
        scores.append(score)
    return scores

# Calculate sentiment scores for the preprocessed data
scores = sentiment_scores_with_tfidf_enhanced(texts, enhanced_sentiment_dict, tfidf_matrix, vectorizer)

print("\nSentiment Scores with Enhanced Dictionary:")
for text, score in zip(texts, scores):
    print(f"Text: {text}, Score: {score}")


Sentiment Scores with Enhanced Dictionary:
Text: tsla gme amc bb bbby back, Score: -0.0762056657644598
Text: gnus gme koss amc tsla lets shows fuckers well win battle get intimidated loss weve making gains month lol stocks game thrones type month, Score: -0.007294882869555803
Text: amc anyone selling right fckin idiot hope realize tomorrow get fcked gon na regret im loading takin shares good luck bulls gon na make pop tomorrow friday tsla spy amzn msft, Score: -0.07224966057572965
Text: sndl share sabby believe trying ruin us tsla gme amc nakd stop selling sndl shares dude share, Score: -0.02439097431800049
Text: tsla listened entire earnings call media dissects information believe see stock pop right back continue fulls team ahead beginning tesla, Score: -0.0727891837202565
Text: tsla could always wrong strange feeling tesla going drop like lead balloon, Score: -0.13581379127880336
Text: spy aapl tsla tomorrow good luck going higher, Score: -0.13593621221735525
Text: time short tsla,